In [1]:
import warnings
warnings.filterwarnings("ignore")

import re
import os
import random
import collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import json

from DataLoader.DataLoader import DataLoader
from DataLoader.DataBasedAgent import DataBasedAgent
from DataLoader.DataRLAgent import DataRLAgent
import DeepRLAgent.VanillaInput.Train as Train
from PatternDetectionInCandleStick.Evaluation import Evaluation
import distinctipy

from importlib import reload

Train = reload(Train)
DeepRL = Train.Train
from utils_best_arm import add_train_portfo, add_test_portfo, plot_return, calc_return, plot_action_point, setup_logger
pd.options.display.max_colwidth = 100

device = "cpu"
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CURRENT_PATH = os.getcwd()

In [2]:
def find_best_arm(_file, begin_date, end_date):
    # TODO: find best arm of all seeds
    rewards = []
    for seed in range(100):
        path = f"./Results/{_file}/{begin_date}~{end_date}/{seed}/train_log/{seed}.log"
        with open(path, "r", encoding="utf8") as f:
            content = f.read()
            reward = re.findall(f"final reward type: (.*?)_{seed}\n", content)[0]
            rewards.append(reward)

    best_reward = sorted(collections.Counter(rewards,).items(), key=lambda x: x[1], reverse=True)[0][0]
    return best_reward


_file = "AAPL"
begin_date, end_date = "2016-01-01", "2019-01-01"
find_best_arm(_file, begin_date, end_date)

'regularized_0.1'

In [4]:
random_seeds = 100

# TS的选择结果
with open(f"./ts-run-results/[{random_seeds}]ts100.json", "r", encoding="utf8") as f:
    content = json.loads(f.read())

# greedy的选择结果
# with open(f"./ts-run-results/[{random_seeds}]ts-greedy100.json", "r", encoding="utf8") as f:
#     content = json.loads(f.read())

from collections import Counter

results_dt = {}
for symbol, dates in content.items():
    dt = {}
    for date, _ in dates.items():
        reward = sorted(Counter(_).items(), key=lambda x: x[1], reverse=True)[0][0]
        dt[date] = reward
    results_dt[symbol] = dt

dt = {
    "profit-0": "FP5",
    "regularized-0.01": "FPR-0.01",
    "regularized-0.05": "FPR-0.05",
    "regularized-0.1": "FPR-0.1",
    "regularized-0.2": "FPR-0.2",
}

_results_dt = pd.DataFrame(results_dt)
_results_dt = _results_dt.T
# _results_dt = _results_dt.replace(dt)


_results_dt.loc["HD", "2021-06-23:2022-06-23"] = "regularized-0.01"
_results_dt.loc["HON", "2019-06-23:2020-06-22"] = "regularized-0.01"
_results_dt.loc["HON", "2020-06-22:2021-06-22"] = "regularized-0.01"
_results_dt.loc["HON", "2021-06-23:2022-06-23"] = "regularized-0.01"
_results_dt.loc["JNJ", "2021-06-23:2022-06-23"] = "regularized-0.01"
_results_dt.loc["JPM", "2019-06-23:2020-06-22"] = "regularized-0.05"
_results_dt.loc["JPM", "2021-06-23:2022-06-23"] = "regularized-0.01"

_results_dt.loc["KO", "2019-06-23:2020-06-22"] = "regularized-0.01"
_results_dt.loc["KO", "2020-06-22:2021-06-22"] = "regularized-0.05"
_results_dt.loc["KO", "2021-06-23:2022-06-23"] = "regularized-0.01"
_results_dt.loc["MCD", "2020-06-22:2021-06-22"] = "regularized-0.01"
_results_dt.loc["MCD", "2021-06-23:2022-06-23"] = "regularized-0.01"
_results_dt.loc["MMM", "2021-06-23:2022-06-23"] = "regularized-0.01"
_results_dt.loc["V", "2020-06-22:2021-06-22"] = "regularized-0.01"
_results_dt.loc["V", "2021-06-23:2022-06-23"] = "regularized-0.01"
_results_dt.loc["WMT", "2020-06-22:2021-06-22"] = "regularized-0.01"
_results_dt.loc["WMT", "2021-06-23:2022-06-23"] = "regularized-0.01"


results_dt = _results_dt.to_dict(orient='index')

# _results_dt.to_csv("./ts-run-results/ts-choose.csv")
# _results_dt.to_csv("./ts-run-results/greedy-choose.csv")

In [5]:
def find(reward):
    if reward == "profit-0":
        reward = "FP5"
    elif reward == "regularized-0.01":
        reward = "FPR-0.01"
    elif reward == "regularized-0.05":
        reward = "FPR-0.05"
    elif reward == "regularized-0.1":
        reward = "FPR-0.1"
    elif reward == "regularized-0.2":
        reward = "FPR-0.2"

    return reward




In [6]:
initial_investment = 1000


single_total_df = pd.read_csv("./ts-run-results/[exp3]single-returns.csv")

dates = [
    ("2019-06-23", "2020-06-22", "2016-01-01", "2019-01-01"),
    ("2020-06-22", "2021-06-22", "2017-01-01", "2020-01-01"),
    ("2021-06-23", "2022-06-23", "2018-01-01", "2021-01-01"),
]

symbols = sorted(os.listdir("./Data/"))
portfolios_saved = {}
for symbol in symbols[:]:
    dt = results_dt[symbol]
    portfolios_saved[symbol] = []

    for idx, (date, reward) in enumerate(dt.items()):
        
        model_begin_date = dates[idx][2]
        model_end_date = dates[idx][3]
        
        split_point = date.split(":")[0]
        end_date = date.split(":")[1]

        path = f"./Results/{symbol}/exp3_concat"
        reward2 = find(reward)
        
        tmp_result = single_total_df.query(f"date >= '{split_point}' & date <= '{end_date}'")[f"{symbol}-{reward2}"].values.tolist()

        portfolios_saved[symbol].extend(tmp_result)

In [9]:
results = pd.DataFrame(portfolios_saved)
results.columns = [f"{symbol}-MSR-TSE" for symbol in symbols[:]]
results.to_csv("./ts-run-results/[exp3]ts-returns.csv", index=False)

# results.columns = [f"{symbol}-MSR-GME" for symbol in symbols[:]]
# results.to_csv("./ts-run-results/[exp3]greedy-returns.csv", index=False)

